## 🎓 Persona : Léa, jeune investisseuse étudiante

**Profil :**
- 👩 24 ans, diplômée de l'EM Lyon
- 💼 Première expérience professionnelle après 2 ans d'alternance
- 💰 Aide parentale pour le financement + épargne personnelle (~15 000 €)
- 🎯 Objectif : réaliser un **premier investissement locatif** dans une **ville étudiante dynamique**

---

### 💡 Objectif d'investissement
> Trouver le **meilleur investissement locatif étudiant** possible avec un **budget global de 200 000 €**,  
> en analysant la rentabilité brute dans les **principales villes étudiantes françaises** (studios et T1 ≤45m²)

---

### 💰 Hypothèses financières
| Élément | Montant estimé |
|----------|----------------|
| Prix d'achat visé | 160 000 – 180 000 € |
| Apport personnel | 15 000 € |
| Prêt immobilier estimé | 180 000 € sur 20 ans |
| Budget total (frais inclus) | **≈ 200 000 €** |
| Objectif de rentabilité brute | **≥ 5 %** |

---

### 🔍 Besoins data de Léa
- Évaluer le **taux de vacance locative** en France pour anticiper les périodes creuses (notamment l'été où les étudiants quittent les logements)
- Visualiser les **villes à forte concentration étudiante** en France
- Analyser l'**évolution du prix au m² à l'achat et des loyers étudiants** en France
- Etudier **la rentabilité moyenne en France** en 2024
- Analyser la **dynamique du marché immobilier local : croissance ou baisse des prix et loyers sur les 5 dernières années** (entre Rennes et Bordeaux)
- Comparer **les quartiers les plus rentables (rentabilité brute)** à ?
- Analyser la **localisation/nombre des transports en commun** pour identifier les zones les plus attractives pour les étudiants à ?
- Analyser la **localisation des universités/grandes écoles** à ?
- Analyser les **quartiers vivants (nombre de resto, bars, et supermarchés)** à ? 
- Fournir une **recommandation finale : "où investir avec 200k€ ?"**

---

### 🧭 Objectif du notebook
Créer un outil interactif permettant à Léa de :
1. Analyser la **rentabilité locative brute** pour appartements étudiants ≤45m² dans **23 grandes villes françaises**
2. Explorer visuellement les **villes à forte concentration étudiante** et analyser les **taux de vacance locative**
3. Obtenir un **classement des villes** par rentabilité, prix et loyers pour décider où investir avec 200k€

## 📚 Import des bibliothèques ##

In [ ]:
import pandas as pd 
import plotly.express as px

# 2e vision : Analyse resserrée sur des villes #

### Analyser la **dynamique du marché immobilier local : croissance ou baisse des prix et loyers sur les 5 dernières années** (entre Rennes et Bordeaux) ###

In [8]:
print("Partie Axel")

Partie Axel


### Comparer **les quartiers les plus rentables (rentabilité brute)** à ? ###

In [9]:
print("Partie Lucien")

Partie Lucien


### Analyser la **localisation/nombre des transports en commun** pour identifier les zones les plus attractives pour les étudiants à ?  ###

In [12]:
print("Partie Valentine")

Partie Valentine


### Analyser la **localisation des universités/grandes écoles** à ? ###

In [42]:
url_df_enseignement_sup = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/fr-esr-atlas_regional-effectifs-d-etudiants-inscrits-detail_etablissements.csv"

df_enseignement_sup = pd.read_csv(url_df_enseignement_sup, delimiter=';')

df_enseignement_sup.head()

,Rentrée universitaire,catégorie d'établissement,secteur d'établissement,code UAI de l'établissement,sigle de l'établissement,libellé de l'établissement,libellé complémentaire de l'établissement,Code UAI de la composante,sigle de la composante,libellé de la composante,...,id unité urbaine,unité urbaine,code commune,Commune,gps,degetu,degre_etudes,nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE,dont femmes,dont hommes
0,2023,"Écoles de commerce, gestion et vente",Privé,0562044T,EC TECH SUP PR,GROUPE AFTEC,IPAC BACHELOR FACTORY VANNES,0562044T,EC TECH SUP PR,GROUPE AFTEC,...,UU56501,Vannes,56158,Plescop,"47.67764394859038, -2.80211043128016",3.0,BAC + 3,112,96,16
1,2023,"Écoles de commerce, gestion et vente",Privé,0590346F,SKEMA LILLE,SKEMA BUSINESS SCHOOL LILLE,NaN,0060656F,E.COM,ECOLE DE COMMERCE,...,UU06701,Nice,06152,Valbonne,"43.61328883276337, 7.056290282560445",6.0,BAC + 6 et plus,19,9,10
2,2023,"Écoles de commerce, gestion et vente",Privé,0590346F,SKEMA LILLE,SKEMA BUSINESS SCHOOL LILLE,NaN,216P0001,SKEMA CHINE,SKEMA BUSINESS SCHOOL,...,CHI99216,Suzhou,99216,Suzhou,NaN,4.0,BAC + 4,389,67,322
3,2023,"Écoles de commerce, gestion et vente",Privé,0590350K,EDHEC LILLE,EDHEC BUSINESS SCHOOL,MEMBRE FUPL,0755719J,EDHEC,CENTRE DE FORMATION CONTINUE,...,UU00851,Paris,75102,Paris 2e,NaN,5.0,BAC + 5,387,162,225
4,2023,"Écoles de commerce, gestion et vente",Privé,0593202K,IESEG LILLE,INST ECO SCIENT GESTION LILLE,MEMBRE FUPL,0593202K,IESEG LILLE,INST ECO SCIENT GESTION LILLE,...,UU59702,Lille,59350,Lille,"50.63553116922688, 3.044909976114497",2.0,BAC + 2,739,285,454


In [46]:
df_rennes_et_bordeaux = df_enseignement_sup[
    df_enseignement_sup['Commune'].isin(['Bordeaux', 'Rennes'])
]

df_rennes_et_bordeaux = df_rennes_et_bordeaux.dropna(subset=['gps'])

print(f"Nombres de lignes sans gps à Rennes et Bordeaux :", df_rennes_et_bordeaux['gps'].isnull().sum())
df_rennes_et_bordeaux.head()

Nombres de lignes sans gps à Rennes et Bordeaux : 0


,Rentrée universitaire,catégorie d'établissement,secteur d'établissement,code UAI de l'établissement,sigle de l'établissement,libellé de l'établissement,libellé complémentaire de l'établissement,Code UAI de la composante,sigle de la composante,libellé de la composante,...,id unité urbaine,unité urbaine,code commune,Commune,gps,degetu,degre_etudes,nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE,dont femmes,dont hommes
25,2023,Écoles d'ingénieurs,Privé,0922455U,CESI EI NANTER,CESI ECOLE INGENIEURS NANTERRE,CESI PARIS,0332984P,CESI EI BDEAUX,CESI ECOLE INGENIEURS BORDEAUX,...,UU33701,Bordeaux,33063,Bordeaux,"44.864981775219746, -0.5775733061462319",2.0,BAC + 2,15,5,10
110,2023,Lycées,Public,0330023W,LGT,LYCEE GENERAL ET TECHNOLOGIQUE,CAMILLE JULLIAN,0330023W,LGT,LYCEE GENERAL ET TECHNOLOGIQUE,...,UU33701,Bordeaux,33063,Bordeaux,"44.844623375958385, -0.5902035283353784",1.0,BAC + 1,204,112,92
322,2022,Lycées,Privé,0352267Z,ECP PR,ECOLE PRIVEE COIF ESTHETIQUE,L'ACADEMY - SILVYA TERRADE REN,0352267Z,ECP PR,ECOLE PRIVEE COIF ESTHETIQUE,...,UU35701,Rennes,35238,Rennes,"48.10427073898263, -1.6691603554365282",2.0,BAC + 2,9,8,1
367,2022,Lycées,Public,0332468D,LT LYC METIER,LT LYCEE DES METIERS,BIOLOGIE ET CHIMIE ST LOUIS,0332468D,LT LYC METIER,LT LYCEE DES METIERS,...,UU33701,Bordeaux,33063,Bordeaux,"44.866484243847566, -0.5674100961785374",1.0,BAC + 1,137,99,38
565,2023,"Écoles de commerce, gestion et vente",Privé,0332524P,INSEEC BORDX,INST HTES ETUDES ECO COM BORDX,GROUPE OMNES EDUCATION,0332524P,INSEEC BORDX,INST HTES ETUDES ECO COM BORDX,...,UU33701,Bordeaux,33063,Bordeaux,"44.859124286097206, -0.5557964100719849",3.0,BAC + 3,29,14,15


In [59]:
# Nettoyage des coordonnées
df_rennes_et_bordeaux[['lat', 'lon']] = df_rennes_et_bordeaux['gps'].str.split(',', expand=True)
df_rennes_et_bordeaux['lat'] = df_rennes_et_bordeaux['lat'].astype(float)
df_rennes_et_bordeaux['lon'] = df_rennes_et_bordeaux['lon'].astype(float)

# On enlève les lignes sans coordonnées
df_rennes_et_bordeaux = df_rennes_et_bordeaux.dropna(subset=['lat', 'lon'])

df_rennes = df_rennes_et_bordeaux[df_rennes_et_bordeaux['Commune'] == 'Rennes']

fig_rennes = px.scatter_map(
    df_rennes,
    lat="lat",
    lon="lon",
    color="catégorie d'établissement",  # tu peux changer par 'secteur d'établissement' ou autre
    size="nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE",
    hover_name="libellé de l'établissement",
    hover_data={
        "Commune": True,
        "nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE": True,
        "secteur d'établissement": True,
        "degre_etudes": True
    },
    zoom=12,
    height=600,
    center={"lat": 48.117, "lon": -1.677}
)

fig_rennes.update_layout(
    mapbox_style="open-street-map",  # carte claire
    title="Répartition géographique des établissements d’enseignement supérieur en France (2023)",
    margin={"r":0,"t":50,"l":0,"b":0}
)

fig_rennes.show()

In [61]:
df_bordeaux = df_rennes_et_bordeaux[df_rennes_et_bordeaux['Commune'] == 'Bordeaux']

fig_bordeaux = px.scatter_map(
    df_bordeaux,
    lat="lat",
    lon="lon",
    color="catégorie d'établissement",
    size="nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE",
    hover_name="libellé de l'établissement",
    hover_data={
        "Commune": True,
        "nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE": True,
        "secteur d'établissement": True,
        "degre_etudes": True
    },
    zoom=12,  # zoom sur Bordeaux
    height=600,
    center={"lat": 44.8378, "lon": -0.5792}  # centre sur Bordeaux
)

fig_bordeaux.update_layout(
    mapbox_style="open-street-map",
    title="Établissements d'enseignement supérieur à Bordeaux (2023)",
    margin={"r":0,"t":50,"l":0,"b":0}
)

fig_bordeaux.show()


In [70]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Filtrer les données
df_rennes = df_rennes_et_bordeaux[df_rennes_et_bordeaux['Commune'] == 'Rennes']
df_bordeaux = df_rennes_et_bordeaux[df_rennes_et_bordeaux['Commune'] == 'Bordeaux']

# Créer une figure avec 1 ligne et 2 colonnes
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Rennes", "Bordeaux"),
    specs=[[{"type": "scattermapbox"}, {"type": "scattermapbox"}]]
)

# Rennes
fig.add_trace(
    go.Scattermapbox(
        lat=df_rennes['lat'],
        lon=df_rennes['lon'],
        mode='markers',
        marker=dict(
            size=df_rennes['nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE'] / 2,  # taille relative
            color=df_rennes["secteur d'établissement"].map({"Privé": "blue", "Public": "green"}),
            sizemode='area',
            sizemin=5
        ),
        text=df_rennes["libellé de l'établissement"],
        hovertemplate="<b>%{text}</b><br>Étudiants: %{marker.size}<br>Secteur: %{marker.color}<extra></extra>"
    ),
    row=1, col=1
)

# Bordeaux
fig.add_trace(
    go.Scattermapbox(
        lat=df_bordeaux['lat'],
        lon=df_bordeaux['lon'],
        mode='markers',
        marker=dict(
            size=df_bordeaux['nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE'] / 2,
            color=df_bordeaux["secteur d'établissement"].map({"Privé": "blue", "Public": "green"}),
            sizemode='area',
            sizemin=5
        ),
        text=df_bordeaux["libellé de l'établissement"],
        hovertemplate="<b>%{text}</b><br>Étudiants: %{marker.size}<br>Secteur: %{marker.color}<extra></extra>"
    ),
    row=1, col=2
)

# Layout
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center={"lat": 48.117, "lon": -1.677},  # centre par défaut, ne s'applique pas aux sous-cartes individuellement
        zoom=12
    ),
    mapbox2=dict(
        style="open-street-map",
        center={"lat": 44.8378, "lon": -0.5792},
        zoom=12
    ),
    height=600,
    title_text="Établissements d'enseignement supérieur à Rennes et Bordeaux (Privé/Public)"
)

fig.show()


/var/folders/v0/_lflz19n6hv63x17d4_zjfjm0000gn/T/ipykernel_76510/2379515828.py:17: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/var/folders/v0/_lflz19n6hv63x17d4_zjfjm0000gn/T/ipykernel_76510/2379515828.py:35: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



### Analyser les **quartiers vivants (nombre de resto, bars, et supermarchés)** à ? ###

In [10]:
print("Partie Adam et Valentine")

Partie Adam et Valentine
